In [7]:
import time
import numpy as np
import h5py
import hdf5plugin
import pandas as pd
import matplotlib as mpl
import matplotlib.colors as cm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from unyt import yr, Myr, kpc, arcsec, nJy, Mpc, Msun, erg, s, Hz, kelvin
from astropy.cosmology import Planck18 as cosmo
from scipy import signal
from synthesizer.emission_models.attenuation import PowerLaw
from synthesizer.filters import UVJ
from synthesizer.grid import Grid
from synthesizer.load_data.load_camels import load_CAMELS_IllustrisTNG
from synthesizer.sed import Sed
from synthesizer.parametric import SFH, ZDist
from synthesizer.particle.stars import sample_sfhz
from synthesizer.parametric import Stars as ParametricStars
from synthesizer.particle.particles import CoordinateGenerator
from synthesizer.filters import Filter, FilterCollection
from synthesizer.sed import combine_list_of_seds
from synthesizer.kernel_functions import Kernel
from synthesizer.conversions import lnu_to_absolute_mag
from synthesizer.emission_models.attenuation import PowerLaw
from synthesizer.emission_models.dust.emission import Blackbody, Greybody
from synthesizer.emission_models import (
    EmissionModel,
    AttenuatedEmission,
    BimodalPacmanEmission,
    DustEmission,
    EmissionModel,
    UnifiedAGN,
    CharlotFall2000,
    IncidentEmission,
    NebularEmission,
    ReprocessedEmission,
    StellarEmissionModel,
)
from synthesizer.conversions import lnu_to_absolute_mag, fnu_to_apparent_mag, fnu_to_lnu
grid_name = "bc03_chabrier03-0.1,100.hdf5"
grid_dir = "/home/jovyan/"
grid = Grid(grid_name, grid_dir=grid_dir, read_lines=False)
CV_X = 'CV_0'
dir_ = '/home/jovyan/Data/Sims/IllustrisTNG/CV/' + CV_X
gals_074 = load_CAMELS_IllustrisTNG(
    dir_,
    snap_name='snapshot_074.hdf5', 
    group_name='groups_074.hdf5',
)
gals_074 = [gal for gal in gals_074 if len(gal.stars.ages) >= 100]  
# Filter galaxies to only include those with 100 or more star particles
gals_074 = [gal for gal in gals_074 if gal.stars.nstars >= 100]
print(len(gals_074))
gals_074 = [gal for gal in gals_074 if gal.gas.nparticles >= 100]


370


array([ -3.48814625,  -3.5912223 ,  -3.65771917, ..., -10.48283535,
       -10.1445652 ,  -9.85118686])

In [8]:

# get sprectra, incident model atm
incident = IncidentEmission(grid)
little_h =  0.6711
spec_list = []


In [9]:
# Lets work with z=0 so gals_025
for i in gals_074:
    gal = i
    # get_spectra_incident An Sed object containing the stellar spectra
    spec = gal.stars.get_spectra(incident)
    spec.get_fnu0()
    spec_list.append(spec)


In [10]:
# combine
seds = combine_list_of_seds(spec_list)
seds
spec.lnu  # get luminosity


unyt_array([1.07936779e+22, 1.18686104e+22, 1.26182353e+22, ...,
       6.77711873e+24, 4.96291724e+24, 3.78778832e+24], 'erg/(Hz*s)')

In [11]:
filt1 = Filter("top_hat/filter.1", lam_min=1400, lam_max=1600, new_lam=grid.lam)
filt_lst = [filt1]#, fil_uvm2_XMM, fil_uvm2_UVOT]
combined_filters = FilterCollection(
    filters=filt_lst, new_lam=grid.lam
)
spec.get_photo_lnu(combined_filters, verbose=True) 

abs_mag = lnu_to_absolute_mag(spec.lnu)
abs_mag

array([ -3.48814625,  -3.5912223 ,  -3.65771917, ..., -10.48283535,
       -10.1445652 ,  -9.85118686])